The goal is to do some brief analysis of the dataset before running an expirement. For the expirement, I will choose a recommendation algorithm from both SurpRise and Tensorflow Recommenders packages. I will run the analysis once on CPU and then again on GPU to compare both the scores between the two different models along with the time differences between CPU and GPU.

In [2]:
!pip3 install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.8 MB 4.3 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633705 sha256=8474922e3d2ffd56dda198526e1f0ca03ba52783715144bfc2eddffaf3d0a5a4
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [3]:
import numpy as np
import pandas as pd

import os
import csv
from collections import defaultdict

from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import cross_validate



## Data Ingest

In [4]:
data = pd.read_csv('https://raw.githubusercontent.com/jhancuch/onmart/main/recommenders/Transactions.csv')

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16372 entries, 0 to 16371
Data columns (total 22 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   CustomerID               16372 non-null  object
 1   FirstName                16372 non-null  object
 2   LastName                 16372 non-null  object
 3   CreditCardNumber         16372 non-null  object
 4   OrderID                  16372 non-null  object
 5   Customer_Occupation      16372 non-null  object
 6   Credit_Card_Number       16372 non-null  int64 
 7   Order_ID                 16372 non-null  int64 
 8   Order_Date               16372 non-null  object
 9   Expected_Delivery_Date   16372 non-null  object
 10  Actual_Delivery_Date     16372 non-null  object
 11  Product_ID               16372 non-null  object
 12  Product_Name             16372 non-null  object
 13  Category                 16372 non-null  object
 14  Manufacturer             16372 non-nul

In [6]:
data.head()

,CustomerID,FirstName,LastName,CreditCardNumber,OrderID,Customer_Occupation,Credit_Card_Number,Order_ID,Order_Date,Expected_Delivery_Date,...,Product_Name,Category,Manufacturer,Review_Rating,Delivery_Tracking_ID,Delivery_Type,Delivery_Zip_Code,Transaction_Status,Order_Returned,Order_Delivered_on_Time
0,566-26-0321,Rebeka,Stokes,xxxx-xxxx-xxxx-2650,50787284-d78d-4b6a-b93f-7419b0d46590,Lawyer,12345789032,98,4/7/2020,4/11/2020,...,Fitbit small,fitness,Fitbit small,3,98,Pickup,10048,Disputed,1,1
1,274-18-0017,Rahsaan,Koch,xxxx-xxxx-xxxx-5055,f61639d7-28de-4507-a63a-71e99921d8f9,Lawyer,12345789032,131,5/10/2020,5/14/2020,...,Fitbit small,fitness,Fitbit small,3,131,Pickup,10048,Disputed,1,1
2,455-28-7950,Zion,Osinski,xxxx-xxxx-xxxx-0150,7b1caf69-bd5b-441c-9115-6fa31ba1c313,Lawyer,12345789032,318,11/13/2020,11/17/2020,...,Fitbit small,fitness,Fitbit small,3,318,Pickup,10048,Disputed,1,1
3,254-72-5938,Ward,Brekke,xxxx-xxxx-xxxx-5280,cb58e8e9-572b-4d9e-af9f-dbecf9dfd3c5,Actor,12345789032,528,2/12/2020,2/16/2020,...,Fitbit large,fitness,Fitbit large,5,528,Pickup,10048,Disputed,0,1
4,563-47-8401,Terrence,Franecki,xxxx-xxxx-xxxx-1019,a9d66460-a48f-419f-8d39-7818290a3041,Actor,12345789032,561,2/12/2020,2/16/2020,...,Fitbit large,fitness,Fitbit large,5,561,Pickup,10048,Disputed,0,1


In [7]:
data.tail()

,CustomerID,FirstName,LastName,CreditCardNumber,OrderID,Customer_Occupation,Credit_Card_Number,Order_ID,Order_Date,Expected_Delivery_Date,...,Product_Name,Category,Manufacturer,Review_Rating,Delivery_Tracking_ID,Delivery_Type,Delivery_Zip_Code,Transaction_Status,Order_Returned,Order_Delivered_on_Time
16367,564-40-9827,Madie,Mayert,xxxx-xxxx-xxxx-1592,1b69b9ee-3017-429e-9e1b-9994ead619c0,Lawyer,12345789032,11949,6/7/2019,6/11/2019,...,Fitbit small,fitness,Fitbit small,3,11949,Pickup,90774,Disputed,1,1
16368,627-64-3054,Vincent,Ondricka,xxxx-xxxx-xxxx-9291,27f2841e-27cd-42d0-8000-f99b7ad7711d,Lawyer,12345789032,12033,8/30/2019,9/3/2019,...,Fitbit small,fitness,Fitbit small,3,12033,Pickup,90774,Disputed,1,1
16369,564-40-9827,Madie,Mayert,xxxx-xxxx-xxxx-1592,46db23f8-a2aa-48e1-96c4-9c07e89ad8eb,Lawyer,12345789032,12236,2/25/2019,3/1/2019,...,Fitbit small,fitness,Fitbit small,3,12236,Pickup,90774,Disputed,1,1
16370,416-42-9023,Lonzo,Tromp,xxxx-xxxx-xxxx-8745,dbffba44-ac7c-41d5-adfb-5298be07d057,Actor,12345789032,12293,1/21/2019,1/25/2019,...,Fitbit large,fitness,Fitbit large,5,12293,Pickup,90774,Disputed,0,1
16371,177-25-9819,Bo,Veum,xxxx-xxxx-xxxx-7679,e05ea8c5-027d-4db3-ba6b-666acab5f614,Lawyer,12345789032,12360,3/1/2019,3/5/2019,...,Fitbit small,fitness,Fitbit small,3,12360,Pickup,90774,Disputed,1,1


## EDA

#### Filter the products with more than 100 ratings.

#### Filter the customers that have rated more than 10 products.

#### Filter the products with more than 100 ratings in a certain city.

#### Filter the customers that have rated more than 10 products in a certain zip-code.

#### Exclude products that are in the top 5 highest product returns from the dataset used to build the recommender system.